# How the Interactive Runner Works with BMTK Configs

The `Profiler.interactive_runner()` method provides an interactive GUI for single-cell electrophysiological profiling using BMTK (Brain Modeling Toolkit) configuration files. This section describes how it reads and processes BMTK configs to populate the template dropdown with available cell models along with the asumptions made by the profiler interactive runner.

## Initialization and Config Loading

When initializing the `Profiler` with a `config` parameter:

```python
profiler = Profiler(config='simulation_config.json')
```

The profiler loads the BMTK simulation configuration file and extracts key paths for mechanisms and templates.

## BMTK Config Structure

### simulation_config.json
The main simulation config contains a `"network"` key pointing to the circuit configuration:

```json
{
  "network": "circuit_config.json",
  "manifest": {
    "$BASE_DIR": "${configdir}"
  }
}
```

### circuit_config.json
The circuit config defines the network structure and component directories:

```json
{
  "manifest": {
    "$BASE_DIR": "${configdir}",
    "$COMPONENTS_DIR": "$BASE_DIR/components"
  },
  "components": {
    "mechanisms_dir": "$COMPONENTS_DIR/mechanisms/modfiles",
    "templates_dir": "$COMPONENTS_DIR/templates"
  },
  "networks": {
    "nodes": [
      {
        "node_types_file": "$NETWORK_DIR/bio_net_node_types.csv",
        "nodes_file": "$NETWORK_DIR/bio_net_nodes.h5"
      }
    ]
  }
}
```

## Path Resolution

1. **Mechanisms Directory**: `$COMPONENTS_DIR/mechanisms/modfiles`
   - Contains NEURON mechanism files (.mod) compiled into shared libraries
   - Loaded via `neuron.load_mechanisms(mechanisms_dir)`

2. **Templates Directory**: `$COMPONENTS_DIR/templates`
   - Contains HOC files defining cell templates (e.g., `templates.hoc`)
   - All `.hoc` files in this directory are loaded into NEURON via `h.load_file()`

3. **Node Types File**: `$NETWORK_DIR/bio_net_node_types.csv`
   - CSV file with cell type properties
   - Must include a `model_template` column specifying NEURON template names

## Template Discovery Process

1. **Load HOC Templates**: All `.hoc` files in `templates_dir` are loaded into NEURON, defining template classes like `CP_Cell`, `FSI_Cell`

2. **Extract Template Names**: 
   - Reads node types from the CSV files specified in `circuit_config.json`
   - Looks for the `model_template` column in the node type data
   - Collects unique template names (e.g., `CP_Cell`, `FSI_Cell`)

3. **Populate Dropdown**: The interactive runner uses these template names to populate the "Template" dropdown menu

## Node Types CSV Format

The node types CSV must be space-separated with this structure:

```
node_type_id model_type morphology pop_name model_template
100 biophysical PN  CP_Cell
101 biophysical FSI  FSI_Cell
```

- `node_type_id`: Unique identifier for the cell type
- `model_type`: "biophysical" for NEURON-based cells
- `morphology`: Path to morphology file (can be relative)
- `pop_name`: Population name (optional)
- `model_template`: **Required** - NEURON template class name defined in the HOC files

In [1]:
from bmtool.singlecell import Profiler

profiler = Profiler(config='simulation_config.json')
profiler.interactive_runner()

--No graphics will be displayed.


loading /Users/gregglickert/Documents/GitHub/bmtool/docs/examples/notebooks/single_cell/components/templates/templates.hoc


Output(layout=Layout(border_bottom='1px solid #ccc', border_left='1px solid #ccc', border_right='1px solid #cc…